<a href="https://colab.research.google.com/github/hahaamg/Generative_AI/blob/main/Week_7/%E5%BA%95%E5%B1%A4%E9%82%8F%E8%BC%AF%E6%80%9D%E8%80%83%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 底層邏輯思考機器人

## 安裝並執行 Ollama

執行安裝 Ollama 的腳本

In [4]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


### Ollama Server
`nohup` "No Hangup"（不掛起）的縮寫，讓程序不會因為退出終端（Shell）而終止。

`ollama serve` 啟動 Ollama 伺服器



In [107]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


### 下載拉取模型

In [6]:
!ollama pull gemma3:4b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pullin

## OpenAI API

因為 ChatGPT 大概是最早紅的大型語言模型，因此許多大型語言模型，都和 OpenAI API 相容，Ollama 也不例外

In [108]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰，但我們沒有真的要用 OpenAI 的服務，金鑰就亂打一通就好

In [109]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式，只是這裡多了 API 服務的網址

預設服務 `port` 是 `11434`。

In [110]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

## 測試 response

了解 `response` 的格式

In [85]:
prompt = "你好!"

In [86]:
response = client.chat.completions.create(
  model="gemma3:4b",
  messages=[
        {"role": "system", "content": "你是一個友善且樂於助人的 AI 助手。請用台灣習慣的中文回應。"},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

嗨嗨！你好呀！今天有什麼我可以幫你的嗎？😊



In [87]:
response

ChatCompletion(id='chatcmpl-842', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='嗨嗨！你好呀！今天有什麼我可以幫你的嗎？😊\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1743657502, model='gemma3:4b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=17, prompt_tokens=37, total_tokens=54, completion_tokens_details=None, prompt_tokens_details=None))

## 設計有記憶的聊天機器人

角色（role）有三種，分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: LLM 的回應

In [111]:
system = '''
你是一個擅長從底層邏輯拆解問題的智慧助手。你的目標是協助使用者釐清問題本質、看見被忽略的關鍵，並提供具建設性的建議，幫助他們提升思考品質與生產力。
你會從核心假設、結構與邏輯出發，辨識潛在盲點，引導使用者跳脫表面解法。你善用第一性原理、系統思維與策略洞察，提供多角度分析與深度回饋。
你的回應理性、簡潔、有層次，能啟發使用者反思與行動，以繁體中文回答，不要用其他語言回答，不用使用**符號
'''
description = "你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)"
model = "gemma3:4b"

In [73]:
icon = "🤖 "
messages = [{"role":"system", "content":system}]
print(icon + description + '\n')

while True:
    prompt = input('> ')
    if '好的'||'謝謝' in prompt:
        print('再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

🤖 你好，我是你的 AI 好友拍拍機器人, 什麼話都可以跟我聊哦 :)



KeyboardInterrupt: Interrupted by user

### 6. 打造一個你的對話機器人 web app!

In [48]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:00


In [50]:
import gradio as gr

對話機器人 app 設定

In [112]:
title = "底層邏輯思考機器人"
description = "你好，我是你的 AI 深度思考與高效行動打造的智慧聊天機器人，請告訴我最近遇到什麼問題，我會將問題透過底層邏輯拆解、指出盲點，協助你釐清思緒、提升決策品質。無論你正在規劃目標、解決難題、還是尋求突破，我都能用第一性原理與系統思維，帶你看見問題背後的真相。這不只是聊天，更是一場啟動生產力與洞察力的對話！"
system ='''
你是一個擅長從底層邏輯拆解問題的智慧助手。你的目標是協助使用者釐清問題本質、看見被忽略的關鍵，並提供具建設性的建議，幫助他們提升思考品質與生產力。
你會從核心假設、結構與邏輯出發，辨識潛在盲點，引導使用者跳脫表面解法。你善用第一性原理、系統思維與策略洞察，提供多角度分析與深度回饋。
你的回應理性、簡潔、有層次，能啟發使用者反思與行動，以繁體中文回答，不要用其他語言回答，不用使用**符號
'''
model = "gemma3:4b"

In [113]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [114]:
state = gr.State(messages)

In [115]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [116]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://320207bcf59737a162.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
